# Notebook for the experiment of building **DeCaf** (**De**sign **C**l**a**ssi**f**ier)

## Architectural Overview/Design
![alt text](https://raw.githubusercontent.com/alvi2496/DeCaf/master/assets/architectural_overview.png)

## Objective
The main objective is to classify discussions from pull request, issue tracker commit messages and code comments as `design` or `general`. We also want to make the classifier cross project compatible.

## Data Collection
- We intent to have three types of data. One data is to train the `Word Embedding` model. As `Word Embedding` requires structured form of literature, we have used sentences from literatures(ex. papers and books). Also we have restricted our choice of papers and books to only from the Software Engineering domain for keep the context of our `Word Embedding` model restricted to Software Engineering. Our `Word Embedding` model will be used to vectorize our train data.
- We have collected our train data from Stack Overflow questions, answers and comments. We have collected data and classified them as `design` or `general` based on the tag. For example, questions and answers that contain `design-patterns` or `software-design` falls under the class of `design` while data tagged as `javascript` or `django` as classified as `general`

## Data Cleaning
Raw data can have a lot of noise. Specially when scraped from documents of website, it can contain a lot of misinformation in the form of names, punctuations, numbers(ex. years), misspelled and incompleted words. Also it can have a lot of stopwords that can make the model confused. We have removed all this to make our data as clean as possible. After the cleaning process, out data only contains words that are not stopwords, present in the english dictionary and has lenght greater than three.

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Assign Data location
- literature: holds the data for word embedding

In [0]:
literature_file = "/content/drive/My Drive/documents/projects/DeCaf/data/literature.txt"
we_model_file = "/content/drive/My Drive/documents/projects/DeCaf/models/we.bin"

## Create Word Embedding
- Use fasttext for word embedding
1. we take literature.txt as our input data.
2. we train the classifier upsupervised since we just want to group the data according to the similarity.
3. we have used skipgram as we have observe that skipgram models works better with subword information that cbow
4. we are taking words with character number from 4-20. since we are removing every word less than three characters, its not important to take the characters less than 4 characters. Also, the design words seems to be on the bigger side. for ex. reproduceability contains 16 characters. we are considering characters upto 25 characters.
5. We are taking 300 dimension of the words. Also looping for 10 epochs. Both because the training corpus is relatively small.

In [3]:
!pip install fasttext

## Train the `Word Embedding` model and save it as we.bin
- Train and save the model if the model is not present
- Load the model from disk if preselt

In [4]:
import os
from datetime import datetime as dt
import fasttext as ft

if not os.path.exists(we_model_file):
  print(str(dt.now())+' Training Word Embedding model...')
  we_model = ft.train_unsupervised(literature_file, "skipgram", minn=4, \
                                   maxn=25, dim=300, epoch=10)
  we_model.save_model(we_model_file)
  print(str(dt.now())+' Model trained and saved successfully')
else:
  print(str(dt.now())+' Loading Word Embedding model from disk...')
  we_model = ft.load_model(we_model_file)
  print(str(dt.now())+' Model loaded successfully.')

2020-03-29 08:08:57.952429 Loading Word Embedding model from disk...


2020-03-29 08:09:02.100574 Model loaded successfully.


## Playing around with the Word Embedding model

Get the dimention of the model

In [5]:
we_model.get_dimension()

300

Get the words of the model... For a demo... 

In [6]:
we_model.get_words()[0:10]

['software',
 'design',
 'system',
 'data',
 'development',
 'process',
 'code',
 'model',
 'used',
 'systems']

Get the vector of a word

In [7]:
we_model.get_word_vector('Maintainability')[0:20]

array([ 0.49718451,  0.2315044 ,  0.00726337, -0.11509292, -0.30824506,
        0.19385934, -0.05463782,  0.07306018, -0.41316083, -0.1308918 ,
       -0.1589214 ,  0.38056305, -0.12928438, -0.02915404,  0.38863796,
        0.06197078, -0.39293942,  0.32120928,  0.20311803,  0.02899477],
      dtype=float32)

Get the vector of a sentence

In [8]:
vector = we_model.get_sentence_vector("Add performance tracker to active admin jobs")
print(vector.shape)

(300,)


## Read and process **TRAIN** data
- We use three types of data from training the model. We scraped question, answer and comment data from Stack Overflow and tagged them `design` or `general`. The tagging was done automatically based on the original tag of the question. Then we process our data as we did before for our word embedding data to clear noise. We also did some additional processing to our train data. We only took those documents that contains more than 10 words. For the others, we discarded them. After processing, we got 1,00,000 documents(50,000-design, 50,000-general) for `questions.csv`, 40,000 documents(20,000-design, 20,000-general) for `answers.csv` and 60,000 documents(30,000-design, 30,000-general) for `comments.csv`.

- Our train data is completely noise free, stopwords free and long documents of more than 10 words per document. All the data is randomly distributed.

- After processing the data and converting the data in vector with the help of our trained word embedding model, we save the data as .npy file for quick access.

#### Assign train data location


In [0]:
train_question_file = "/content/drive/My Drive/documents/projects/DeCaf/data/train_data/questions.csv"
train_answer_file = "/content/drive/My Drive/documents/projects/DeCaf/data/train_data/answers.csv"
train_comment_file = "/content/drive/My Drive/documents/projects/DeCaf/data/train_data/comments.csv"

#### Read the data with pandas

In [0]:
import pandas as pd
from prettytable import PrettyTable

train_question = pd.read_csv(train_question_file)
train_answer = pd.read_csv(train_answer_file)
train_comment = pd.read_csv(train_comment_file)

#### Explore the train data

In [11]:
print("Question train data")
print(train_question.head())
print("Answer train data")
print(train_answer.head())
print("Comment train data")
print(train_comment.head())

Question train data
                                            question    label
0  control pattern according principle elsewhere ...   design
1  messenger throws error getting errors messenge...  general
2  error cannot find symbol variable trying integ...   design
3  data mapper pattern different repository patte...   design
4  start project using poetry start project using...  general
Answer train data
                                              answer    label
0  just tell solved issue found feed posted file ...  general
1  does actually implement copy cannot implement ...  general
2  following comments included code note will spe...  general
3  looks user present undefined method error need...  general
4  think better natural argue method names explic...   design
Comment train data
                                             comment    label
0  public declared class later library members sa...   design
1  using server definitely needs reasonable chanc...  general
2  defined cr

In [12]:
table = PrettyTable()

table.field_names = ["Dataset Name", "Shape", "# of design data", "# of general data"]

qd = train_question[train_question['label']=='design']
qg = train_question[train_question['label']=='general']

ad = train_answer[train_answer['label']=='design']
ag = train_answer[train_answer['label']=='general']

cd = train_comment[train_comment['label']=='design']
cg = train_comment[train_comment['label']=='general']

table.add_row(["Question", train_question.shape, qd.shape[0], qg.shape[0]])
table.add_row(["Answer", train_answer.shape, ad.shape[0], ag.shape[0]])
table.add_row(["Comment", train_comment.shape, cd.shape[0], cg.shape[0]])


print(table)

+--------------+-------------+------------------+-------------------+
| Dataset Name |    Shape    | # of design data | # of general data |
+--------------+-------------+------------------+-------------------+
|   Question   | (100000, 2) |      50000       |       50000       |
|    Answer    |  (40000, 2) |      20000       |       20000       |
|   Comment    |  (60000, 2) |      30000       |       30000       |
+--------------+-------------+------------------+-------------------+


### Convert sentences in vector uisng Word Embedding model



Now we turn our head towards converting our train data to vectors of 300 dimension. We are naming our variable by the following convension:
- question data = X_Q, question label = Y_Q
- answer data = X_A, answer label = Y_A
- comment data = X_C, comment label = Y_C

#### Location to save/retrive data in vector format

In [0]:
X_Q_url = "/content/drive/My Drive/documents/projects/DeCaf/data/train_data/X_Q.npy"
Y_Q_url = "/content/drive/My Drive/documents/projects/DeCaf/data/train_data/Y_Q.npy"

X_A_url = "/content/drive/My Drive/documents/projects/DeCaf/data/train_data/X_A.npy"
Y_A_url = "/content/drive/My Drive/documents/projects/DeCaf/data/train_data/Y_A.npy"

X_C_url = "/content/drive/My Drive/documents/projects/DeCaf/data/train_data/X_C.npy"
Y_C_url = "/content/drive/My Drive/documents/projects/DeCaf/data/train_data/Y_C.npy"

We define `get_text_vector(data, url)` function to convert text sentences into 300 dimension word vector. We define `get_label_vector(label, url)` to convert the labels into vector.
- look for the .npy file in the disk. if found then load the data into memory.
- if not found then 
  - convert the text and labels into vector
  - save the text and labels as .npy file 

In [0]:
import numpy as np

def get_text_vector(data, data_url):
  if os.path.exists(data_url):
    X = np.load(data_url)
  else:
    X = []
    for sentence in data:
      X.append(we_model.get_sentence_vector(sentence))
    X = np.array(X)
    np.save(data_url, X)
  return X

In [0]:
def get_label_vector(labels, label_url):
  if os.path.exists(label_url):
    Y = np.load(label_url)
  else:
    Y = []
    for label in labels:
      if label == 'design':
        Y.append(1)
      else:
        Y.append(0)
    Y = np.array(Y)
    np.save(label_url, Y)
  return Y

#### Load and inspect the data 

In [16]:
X_Q = get_text_vector(train_question['question'], X_Q_url)
Y_Q = get_label_vector(train_question['label'], Y_Q_url)
print('Shape of X_Q: ', X_Q.shape)
print('Shape of Y_Q: ', Y_Q.shape)

Shape of X_Q:  (100000, 300)
Shape of Y_Q:  (100000,)


In [17]:
X_A = get_text_vector(train_answer['answer'], X_A_url)
Y_A = get_label_vector(train_answer['label'], Y_A_url)
print('Shape of X_A: ', X_A.shape)
print('Shape of Y_A: ', Y_A.shape)

Shape of X_A:  (40000, 300)
Shape of Y_A:  (40000,)


In [18]:
X_C = get_text_vector(train_comment['comment'], X_C_url)
Y_C = get_label_vector(train_comment['label'], Y_C_url)
print('Shape of X_C: ', X_C.shape)
print('Shape of Y_C: ', Y_C.shape)

Shape of X_C:  (60000, 300)
Shape of Y_C:  (60000,)


## Read and process **VALIDATION** data
- We are taking the same approach as above to read, process, convert and save our validation data.
- Validation data contains a mixture of `question`, `answer` and `comment` data
- X_V for the vector of the text, Y_V is the vector of the labels for validation data.

In [0]:
validation_data_file = "/content/drive/My Drive/documents/projects/DeCaf/data/validation_data/validation.csv"

In [0]:
validation_data = pd.read_csv(validation_data_file)

In [21]:
print("Shape: ", validation_data.shape)
print(validation_data.head())

Shape:  (30000, 2)
                                                text    label
0  better handling type issue sort development us...  general
1  custom route dependency injection route define...   design
2  crawler design calling async calling service l...   design
3  page seems freeze triggered event issue based ...  general
4  install package composer require trying compos...  general


In [22]:
print("Number of design data: ", validation_data[validation_data["label"] == "design"].shape[0])
print("number of general data: ", validation_data[validation_data["label"] == "general"].shape[0])

Number of design data:  15000
number of general data:  15000


In [0]:
X_V = get_text_vector(validation_data['text'], "/content/drive/My Drive/documents/projects/DeCaf/data/validation_data/X_V.npy")
Y_V = get_label_vector(validation_data['label'], "/content/drive/My Drive/documents/projects/DeCaf/data/validation_data/Y_V.npy")

In [24]:
print("Shape of X_V: ", X_V.shape)
print("Shape of Y_V: ", Y_V.shape)

Shape of X_V:  (30000, 300)
Shape of Y_V:  (30000,)


## Read and process **TEST** data
- We are taking the same approach as above to read, process, convert and save our validation data.
- Test data contains a mixture of `question`, `answer` and `comment` data
- X_T for the vector of the text, Y_T is the vector of the labels for validation data.

In [0]:
test_data_file = "/content/drive/My Drive/documents/projects/DeCaf/data/test_data/test.csv"

In [0]:
test_data = pd.read_csv(test_data_file)

In [27]:
print("Shape: ", test_data.shape)
print(test_data.head())

Shape:  (30000, 2)
                                                text    label
0  form using submit insert fairly working stuck ...  general
1  using shared drive portal storage sharing clie...   design
2  scale parallel project looking using together ...   design
3  decouple service associated database separate ...   design
4  method does exist trying send mail user know s...  general


In [28]:
print("Number of design data: ", test_data[test_data["label"] == "design"].shape[0])
print("number of general data: ", test_data[test_data["label"] == "general"].shape[0])

Number of design data:  15000
number of general data:  15000


In [0]:
X_T = get_text_vector(test_data['text'], "/content/drive/My Drive/documents/projects/DeCaf/data/test_data/X_T.npy")
Y_T = get_label_vector(test_data['label'], "/content/drive/My Drive/documents/projects/DeCaf/data/test_data/Y_T.npy")

In [30]:
print("Shape of X_T: ", X_T.shape)
print("Shape of Y_T: ", Y_T.shape)

Shape of X_T:  (30000, 300)
Shape of Y_T:  (30000,)


## Experiment with Traditional Data Mining Algorithms
- Before start experimenting with deep learning, we start our experiment with training some traditional data mining algorithms. We are taking the following classifiers with notations and parameter configurations:
  - k-Nearest Neighbors (`knn`)
  - Linear SVM (`lsvm`)
    - C = Regularization Parameter
  - RBF SVM (`rbf_svm`)
    - Kernel coefficient = 2 
    - Regularization parameter = default = 1
  - Gaussian Process (`gp`)
  - Decision Tree (`dt`)
  - Random Forest (`rf`)
  - Logistic Regression (`lr`)
  - Neural Net (`nn`)
  - AdaBoost (`ab`)
  - Naive Bayes (`nb`)
  - QDA (`qda`)


### Libraries

In [0]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression

### Initialization

In [0]:

names = [
      "Nearest Neighbors", 
      "Linear SVM", 
      "RBF SVM", 
      "Gaussian Process",
      "Decision Tree", 
      "Random Forest",
      "Logistic Regression" 
      "Neural Net", 
      "AdaBoost",
      "Naive Bayes", 
      "QDA"
]

model_paths = [
      "/content/drive/My Drive/documents/projects/DeCaf/models/knn.joblib",
      "/content/drive/My Drive/documents/projects/DeCaf/models/lsvm.joblib",
      "/content/drive/My Drive/documents/projects/DeCaf/models/rbf_svm.joblib",
      "/content/drive/My Drive/documents/projects/DeCaf/models/gp.joblib",
      "/content/drive/My Drive/documents/projects/DeCaf/models/dt.joblib",
      "/content/drive/My Drive/documents/projects/DeCaf/models/rf.joblib",
      "/content/drive/My Drive/documents/projects/DeCaf/models/lr.joblib",
      "/content/drive/My Drive/documents/projects/DeCaf/models/nn.joblib",
      "/content/drive/My Drive/documents/projects/DeCaf/models/ab.joblib",
      "/content/drive/My Drive/documents/projects/DeCaf/models/nb.joblib",
      "/content/drive/My Drive/documents/projects/DeCaf/models/qda.joblib"
]

classifiers = [
      KNeighborsClassifier(n_jobs=-1), 
      SVC(kernel="linear", C=0.025), 
      SVC(gamma=2, C=1), 
      GaussianProcessClassifier(1.0 * RBF(1.0)),
      DecisionTreeClassifier(max_depth=5),
      RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
      LogisticRegression(),
      MLPClassifier(alpha=1, max_iter=1000),
      AdaBoostClassifier(),
      GaussianNB(),
      QuadraticDiscriminantAnalysis()
]

### Combine the three train data namely: `questions`, `answers` and `comment`

In [0]:
X = np.concatenate((X_Q, X_A, X_C), axis=0)
Y = np.concatenate((Y_Q, Y_A, Y_C), axis=0)

### Examine X and Y

In [34]:
print(X.shape)
print(Y.shape)

(200000, 300)
(200000,)


### **Train**, **Save** and **Load** the models into memory

In [0]:
from joblib import dump, load

for index, classifier in enumerate(classifiers):
  if os.path.exists(model_paths[index]):
    print(str(dt.now()) + " Started loading model: ", names[index])
    model = load(model_paths[index])
    print(str(dt.now()) + " Finished loading model: ", names[index])
  else:
    print(str(dt.now()) + " Started training model: ", names[index])
    model = classifier.fit(X, Y)
    print(str(dt.now()) + " Finished training model: ", names[index])
    dump(model, model_paths[index])

2020-03-29 08:09:04.708288 Started training model:  Nearest Neighbors
